# Notebook for dataloader, model training and inference 
## This notebook consists of steps to 
1. load processed graph data into a data dict  raw data into edge, node list data for the data loader for model training 
2. load the model training configurations 
3. pass the data dict in step (1) to the model 
4. train the model 
5. apply model inference on the specific snapshot

In [86]:
import sys 
import os

In [2]:
sys.path.append('/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/')
sys.path.append('/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/src/')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pickle

In [21]:
import numpy as np

In [ ]:
! pip install -r ../src/requirements_taddy.txt

# Load processed data 

In [5]:
# open the training data pickle
file = open("../../data/03_primary/training_data.pkl", 'rb')

# load the pickle file 
data = pickle.load(file)

# close the file
file.close()

In [6]:
rows, cols, labels, weights, headtail, train_size, test_size, nb_nodes, nb_edges = data

In [7]:
train_size, test_size, nb_nodes, nb_edges 

(5, 5, 4162, 47132)

In [8]:
type(headtail)

numpy.ndarray

In [9]:
headtail #sparse matrix saving the node indexes of all edges 

array([list([0, 4119, 4127, 4134, 4136, 4146, 4154]),
       list([1, 4121, 4122, 4133, 4140, 4146, 4149, 4151, 4152, 4155]),
       list([2, 4112, 4130, 4134, 4136, 4142, 4155, 4157]), ...,
       list([46, 147, 769, 1100, 1453, 1478, 1539, 1554, 1632, 1654, 1660, 1782, 1917, 1993, 2187, 2387, 2413, 2473, 2594, 2624, 2626, 2772, 2816, 2844, 2903, 2981, 3036, 3104, 3303, 3326, 3412, 3438, 3442, 3488, 3685, 3871, 4159]),
       list([25, 48, 50, 61, 94, 108, 111, 123, 126, 137, 144, 149, 173, 188, 210, 224, 275, 289, 291, 295, 297, 302, 323, 330, 355, 365, 409, 415, 416, 497, 498, 509, 510, 529, 530, 552, 556, 571, 588, 593, 595, 613, 617, 629, 676, 706, 707, 751, 773, 793, 801, 808, 818, 821, 832, 848, 867, 868, 870, 889, 890, 893, 919, 961, 964, 997, 1000, 1007, 1017, 1050, 1064, 1076, 1079, 1080, 1086, 1138, 1141, 1143, 1151, 1156, 1241, 1270, 1281, 1287, 1374, 1385, 1398, 1413, 1416, 1425, 1452, 1507, 1521, 1530, 1534, 1548, 1553, 1557, 1584, 1586, 1607, 1611, 1616, 1617, 1654, 1661

In [10]:
rows

[array([3317, 2363, 3396, ..., 1738, 2754, 2754], dtype=int32),
 array([2431, 2431, 2350, ..., 3564, 2144, 4055], dtype=int32),
 array([ 642, 3586,   67, ..., 2095, 3604, 2196], dtype=int32),
 array([3989, 3989, 1160, ...,  320,  782,  782], dtype=int32),
 array([2515,  255,  985, ..., 2817, 1885, 2495], dtype=int32),
 array([2495, 2495, 1447, ..., 1162, 3497, 3497], dtype=int32),
 array([ 308, 1099, 1099, ...,  225, 2671, 1855], dtype=int32),
 array([1948, 1747, 1381, ..., 1106, 3293, 3859], dtype=int32),
 array([2286, 2291, 2493, ..., 1003, 2395, 2186], dtype=int32),
 array([1384, 1259,  444, ...,  529, 1083, 3304], dtype=int32)]

In [11]:
cols

[array([4148, 4154, 4127, ..., 4132, 4146, 4148], dtype=int32),
 array([4122, 4114, 4146, ..., 4131, 4150, 4142], dtype=int32),
 array([4158, 4131, 4146, ..., 4116, 4136, 4142], dtype=int32),
 array([4116, 4150, 4134, ..., 4117, 4143, 4123], dtype=int32),
 array([4149, 4149, 4127, ..., 4143, 4116, 4139], dtype=int32),
 array([4153, 4160, 4143, ..., 4157, 4112, 4135], dtype=int32),
 array([4152, 4154, 4120, ..., 4130, 4157, 4122], dtype=int32),
 array([4121, 4157, 4116, ..., 4157, 4114, 4157], dtype=int32),
 array([4157, 4157, 4142, ..., 4142, 4130, 4142], dtype=int32),
 array([4130, 4130, 4130, ..., 4143, 4130, 4130], dtype=int32)]

In [22]:
degrees = np.array([len(x) for x in headtail])
num_snap = test_size + train_size
labels = [torch.LongTensor(label) for label in labels]

snap_train = list(range(num_snap))[:train_size]
snap_test = list(range(num_snap))[train_size:]

In [24]:
snap_train

[0, 1, 2, 3, 4]

In [23]:
snap_test

[5, 6, 7, 8, 9]

In [27]:
labels, len(labels), labels[0].shape

([tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 1,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([1, 1, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([1, 0, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0]),
  tensor([0, 0, 0,  ..., 0, 0, 0])],
 10,
 torch.Size([5000]))

In [78]:
idx = list(range(nb_nodes))
index_id_map = {i:i for i in idx}
idx = np.array(idx)

# Define data dictionary

In [66]:
from anomaly_detection_spatial_temporal_data.model.model_config import TaddyConfig

In [53]:
import yaml

In [52]:
model_config_file = '../../conf/base/parameters/taddy.yml'

In [54]:
with open(model_config_file, "r") as stream:
    try:
        mode_config=yaml.safe_load(stream)
        print(mode_config)
    except yaml.YAMLError as exc:
        print(exc)

{'data_load_options': {'c': 0.15, 'eps': 0.001, 'test_size': 0.2, 'random_state': 3, 'batch_size': 256, 'load_all_tag': False, 'neighbor_num': 5, 'window_size': 2, 'compute_s': True}, 'model_options': {'neighbor_num': 5, 'window_size': 2, 'batch_size': 256, 'embedding_dim': 32, 'num_hidden_layers': 2, 'num_attention_heads': 2, 'seed': 1, 'print_feq': 10, 'lr': 0.001, 'weight_decay': '5e-4', 'max_epoch': 100, 'spy_tag': True, 'max_hop_dis_index': 100, 'max_inti_pos_index': 100, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.5, 'attention_probs_dropout_prob': 0.3, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12, 'is_decoder': False, 'save_directory': '/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/data/07_model_output/'}, 'infer_options': {'snap_num': 6}}


In [68]:
eigen_file_name = "/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/data/05_model_input/eigen_tmp.pkl"
data_loader_config = mode_config['data_load_options']

In [69]:
data_loader_config

{'c': 0.15,
 'eps': 0.001,
 'test_size': 0.2,
 'random_state': 3,
 'batch_size': 256,
 'load_all_tag': False,
 'neighbor_num': 5,
 'window_size': 2,
 'compute_s': True}

In [71]:
type(data_loader_config['compute_s'])

bool

In [75]:
import scipy.sparse as sp
from numpy.linalg import inv

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix. (0226)"""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -0.5).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx).dot(r_mat_inv)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation. (0226)"""
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    # adj_np = np.array(adj.todense())
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj_normalized = sparse_mx_to_torch_sparse_tensor(adj_normalized)
    return adj_normalized

def get_adjs(rows, cols, weights, nb_nodes, eigen_file_name, data_loader_config):
    """Generate adjacency matrix and conduct eigenvalue decomposition for node sampling"""
    if not os.path.exists(eigen_file_name):
        generate_eigen = True
        print('Generating eigen as: ' + eigen_file_name)
    else:
        generate_eigen = False
        print('Loading eigen from: ' + eigen_file_name)
        with open(eigen_file_name, 'rb') as f:
            eigen_adjs_sparse = pickle.load(f)
        eigen_adjs = []
        for eigen_adj_sparse in eigen_adjs_sparse:
            eigen_adjs.append(np.array(eigen_adj_sparse.todense()))

    adjs = []
    if generate_eigen:
        eigen_adjs = []
        eigen_adjs_sparse = []

    for i in range(len(rows)):
        adj = sp.csr_matrix((weights[i], (rows[i], cols[i])), shape=(nb_nodes, nb_nodes), dtype=np.float32)
        adjs.append(preprocess_adj(adj))
        if data_loader_config['compute_s']:
            if generate_eigen:
                eigen_adj = data_loader_config['c'] * inv((sp.eye(adj.shape[0]) - (1 - data_loader_config['c']) * adj_normalize(adj)).toarray())
                for p in range(adj.shape[0]):
                    eigen_adj[p,p] = 0.
                eigen_adj = normalize(eigen_adj)
                eigen_adjs.append(eigen_adj)
                eigen_adjs_sparse.append(sp.csr_matrix(eigen_adj))

        else:
            eigen_adjs.append(None)

    if generate_eigen:
        with open(eigen_file_name, 'wb') as f:
            pickle.dump(eigen_adjs_sparse, f, pickle.HIGHEST_PROTOCOL)

    return adjs, eigen_adjs

In [76]:
edges = [np.vstack((rows[i], cols[i])).T for i in range(num_snap)]
adjs, eigen_adjs = get_adjs(rows, cols, weights, nb_nodes, eigen_file_name, data_loader_config)

Generating eigen as: /home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/data/05_model_input/eigen_tmp.pkl


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: RuntimeWarning: divide by zero encountered in power
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in power


In [80]:
data_dict = {
    'X': None, 
    'A': adjs, 
    'S': eigen_adjs, 
    'index_id_map': index_id_map, 
    'edges': edges,
    'y': labels, 
    'idx': idx, 
    'snap_train': snap_train, 
    'degrees': degrees,
    'snap_test': snap_test, 
    'num_snap': num_snap}

# Define model and load model training config 

In [81]:
from anomaly_detection_spatial_temporal_data.model.dynamic_graph import Taddy

In [82]:
model_config = TaddyConfig(mode_config['model_options'])
model_obj = Taddy(data_dict, model_config)

# Train model

In [83]:
learned_result,save_model_path = model_obj.run()

Epoch: 1, loss:0.6878, Time: 21.1835s
Epoch: 2, loss:0.6822, Time: 20.2179s
Epoch: 3, loss:0.6765, Time: 21.5355s
Epoch: 4, loss:0.6659, Time: 21.4160s
Epoch: 5, loss:0.6435, Time: 22.1088s
Epoch: 6, loss:0.6148, Time: 22.6724s
Epoch: 7, loss:0.5665, Time: 22.7436s
Epoch: 8, loss:0.5621, Time: 20.4716s
Epoch: 9, loss:0.4587, Time: 21.8286s
Epoch: 10, loss:0.4462, Time: 20.9353s
Snap: 05 | AUC: 0.6697
Snap: 06 | AUC: 0.6498
Snap: 07 | AUC: 0.6755
Snap: 08 | AUC: 0.6729
Snap: 09 | AUC: 0.8164
TOTAL AUC:0.7132
Epoch: 11, loss:0.3990, Time: 22.5276s
Epoch: 12, loss:0.4039, Time: 21.1181s
Epoch: 13, loss:0.3986, Time: 21.7353s
Epoch: 14, loss:0.3298, Time: 22.1062s
Epoch: 15, loss:0.3896, Time: 21.0532s
Epoch: 16, loss:0.3966, Time: 21.0583s
Epoch: 17, loss:0.3851, Time: 22.4059s
Epoch: 18, loss:0.3546, Time: 22.6783s
Epoch: 19, loss:0.3315, Time: 23.1748s
Epoch: 20, loss:0.3068, Time: 22.7745s
Snap: 05 | AUC: 0.6714
Snap: 06 | AUC: 0.6520
Snap: 07 | AUC: 0.7571
Snap: 08 | AUC: 0.7450
Snap:

# Model training result 

In [84]:
learned_result

{1: {'train_loss': 0.6877960711717606},
 2: {'train_loss': 0.6821642965078354},
 3: {'train_loss': 0.6764988005161285},
 4: {'train_loss': 0.6658555865287781},
 5: {'train_loss': 0.643513485789299},
 6: {'train_loss': 0.6147666275501251},
 7: {'train_loss': 0.5664583593606949},
 8: {'train_loss': 0.5620778948068619},
 9: {'train_loss': 0.458668515086174},
 10: {'train_loss': 0.4461595192551613, 'test_auc': 0.7132268956049801},
 11: {'train_loss': 0.3990139290690422},
 12: {'train_loss': 0.40388356894254684},
 13: {'train_loss': 0.39861707389354706},
 14: {'train_loss': 0.32978464663028717},
 15: {'train_loss': 0.3895721733570099},
 16: {'train_loss': 0.3966144844889641},
 17: {'train_loss': 0.3850824385881424},
 18: {'train_loss': 0.3545578643679619},
 19: {'train_loss': 0.33154749125242233},
 20: {'train_loss': 0.3068309798836708, 'test_auc': 0.6722487877232647},
 21: {'train_loss': 0.32729755342006683},
 22: {'train_loss': 0.3070712983608246},
 23: {'train_loss': 0.3089311718940735},

In [85]:
save_model_path

'/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/data/07_model_output/taddy_model_99.pth'

### load pipeline model training result

In [12]:
file = open("../../data/07_model_output/train_result.pkl", 'rb')
train_result = pickle.load(file)
# close the file
file.close()

In [13]:
train_result

{1: {'train_loss': 0.6881906241178513},
 2: {'train_loss': 0.6814479827880859},
 3: {'train_loss': 0.6744112074375153},
 4: {'train_loss': 0.6597873270511627},
 5: {'train_loss': 0.6489938050508499},
 6: {'train_loss': 0.6022640466690063},
 7: {'train_loss': 0.5824433118104935},
 8: {'train_loss': 0.545023500919342},
 9: {'train_loss': 0.4961806461215019},
 10: {'train_loss': 0.4321019649505615, 'test_auc': 0.6270922835146469},
 11: {'train_loss': 0.43047888576984406},
 12: {'train_loss': 0.3681452348828316},
 13: {'train_loss': 0.37437988817691803},
 14: {'train_loss': 0.3437521904706955},
 15: {'train_loss': 0.41434410959482193},
 16: {'train_loss': 0.37434427440166473},
 17: {'train_loss': 0.2873542085289955},
 18: {'train_loss': 0.3646049425005913},
 19: {'train_loss': 0.34459684044122696},
 20: {'train_loss': 0.3836841285228729, 'test_auc': 0.6978857767624783},
 21: {'train_loss': 0.28783082962036133},
 22: {'train_loss': 0.44203196465969086},
 23: {'train_loss': 0.293928906321525

# Run inference on the specific snapshot 

### load trained model 

In [16]:
import torch
import transformers

In [17]:
model = torch.load('/home/ec2-user/SageMaker/anomaly-detection-spatial-temporal-data/data/07_model_output/taddy_model_59.pth')

In [18]:
type(model)

anomaly_detection_spatial_temporal_data.model.dynamic_graph.Taddy

In [37]:
snap_num = 9

In [38]:
pred = model.predict(snap_num)

Generating embeddings...
Embeddings created!


In [39]:
type(pred)

numpy.ndarray

In [40]:
pred

array([0.0654242 , 0.0654242 , 0.06851925, ..., 0.98270595, 0.06542419,
       0.06851923], dtype=float32)

In [41]:
labels[1]

tensor([0, 0, 0,  ..., 0, 0, 0])

In [42]:
from sklearn import metrics

In [43]:
auc = metrics.roc_auc_score(labels[snap_num],pred)

In [44]:
auc

0.8840151730005671

### load pipeline inference result

In [55]:
inference_result_filepath_pipeline = "../../data/07_model_output/inference_result.npz"

In [56]:
# check if inference result of the pipeline run is exsiting in the file path 
inference_result = np.load(inference_result_filepath_pipeline)

In [57]:
inference_result['data']

array([0.04597111, 0.04597111, 0.5110733 , ..., 0.4481242 , 0.04574715,
       0.29438794], dtype=float32)